# Monte Carlo Simulations with HPXPy

This notebook demonstrates parallel Monte Carlo simulations using HPXPy.

**Topics covered:**
- Estimating Pi using random sampling
- Monte Carlo integration
- Performance comparison with NumPy
- Scaling with sample size

## 1. Setup

In [1]:
import time
import numpy as np
import hpxpy as hpx

# Initialize HPX runtime
hpx.init()
print(f"HPXPy initialized with {hpx.num_threads()} threads")

HPXPy initialized with 12 threads


## 2. Estimating Pi

We estimate Pi by randomly sampling points in a unit square and counting how many fall inside the unit circle.

The ratio of points inside the circle to total points approximates Pi/4.

In [2]:
def estimate_pi_numpy(n_samples):
    """Estimate Pi using NumPy."""
    x = np.random.uniform(0, 1, n_samples)
    y = np.random.uniform(0, 1, n_samples)
    
    # Count points inside unit circle
    inside = np.sum(x**2 + y**2 <= 1)
    return 4 * inside / n_samples

# Quick test
pi_estimate = estimate_pi_numpy(100000)
print(f"NumPy Pi estimate (100K samples): {pi_estimate:.6f}")
print(f"Actual Pi: {np.pi:.6f}")
print(f"Error: {abs(pi_estimate - np.pi):.6f}")

NumPy Pi estimate (100K samples): 3.145520
Actual Pi: 3.141593
Error: 0.003927


In [3]:
def estimate_pi_hpxpy(n_samples):
    """Estimate Pi using HPXPy with parallel operations."""
    x = hpx.random.rand(n_samples)
    y = hpx.random.rand(n_samples)
    
    # Compute distance squared from origin
    dist_sq = x * x + y * y
    
    # Count points inside unit circle
    # Convert boolean mask to float and sum
    inside_mask = dist_sq <= 1.0
    inside_float = hpx.array(inside_mask.to_numpy().astype(np.float64))
    inside = hpx.sum(inside_float)
    
    return 4 * inside / n_samples

# Quick test
pi_estimate = estimate_pi_hpxpy(100000)
print(f"HPXPy Pi estimate (100K samples): {pi_estimate:.6f}")
print(f"Error: {abs(pi_estimate - np.pi):.6f}")

HPXPy Pi estimate (100K samples): 3.153000
Error: 0.011407


## 3. Convergence Analysis

Let's see how the estimate improves with more samples.

In [4]:
sample_sizes = [1000, 10000, 100000, 1000000, 10000000]

print(f"{'Samples':>12} {'Estimate':>12} {'Error':>12} {'Time (ms)':>12}")
print("=" * 52)

for n in sample_sizes:
    start = time.perf_counter()
    estimate = estimate_pi_hpxpy(n)
    elapsed = (time.perf_counter() - start) * 1000
    error = abs(estimate - np.pi)
    
    print(f"{n:>12,} {estimate:>12.6f} {error:>12.6f} {elapsed:>12.2f}")

     Samples     Estimate        Error    Time (ms)
       1,000     3.124000     0.017593         0.64
      10,000     3.107600     0.033993         0.50
     100,000     3.142360     0.000767         2.75
   1,000,000     3.143292     0.001699        18.39


  10,000,000     3.141332     0.000261       217.27


## 4. Monte Carlo Integration

Estimate the integral of sin(x) from 0 to Pi (which equals 2).

In [5]:
def monte_carlo_integrate_sin(n_samples):
    """Integrate sin(x) from 0 to Pi using Monte Carlo."""
    # Random x values in [0, Pi]
    x = hpx.random.rand(n_samples) * np.pi
    
    # Evaluate sin(x) at random points
    y = hpx.sin(x)
    
    # Monte Carlo estimate: (b-a) * mean(f(x))
    return np.pi * hpx.mean(y)

# Test with increasing samples
print("Integrating sin(x) from 0 to Pi (exact result = 2)")
print(f"{'Samples':>12} {'Estimate':>12} {'Error':>12}")
print("-" * 40)

for n in [10000, 100000, 1000000, 10000000]:
    estimate = monte_carlo_integrate_sin(n)
    error = abs(estimate - 2.0)
    print(f"{n:>12,} {estimate:>12.6f} {error:>12.6f}")

Integrating sin(x) from 0 to Pi (exact result = 2)
     Samples     Estimate        Error
----------------------------------------
      10,000     1.986287     0.013713
     100,000     1.996236     0.003764
   1,000,000     1.997429     0.002571


  10,000,000     2.000305     0.000305


## 5. Multi-dimensional Integration

Monte Carlo shines for high-dimensional integrals where grid methods become impractical.

In [6]:
def integrate_sphere_volume(n_samples, dimensions):
    """Estimate volume of unit n-sphere using Monte Carlo."""
    # Generate random points in n-dimensional unit cube [-1, 1]^n
    points = hpx.random.rand(n_samples, dimensions) * 2 - 1
    
    # HPXPy doesn't yet support axis reduction, use numpy for this
    points_np = points.to_numpy()
    dist_sq = np.sum(points_np * points_np, axis=1)
    
    # Count points inside unit sphere
    inside = np.sum(dist_sq <= 1)
    
    # Volume estimate = (cube volume) * (fraction inside)
    cube_volume = 2 ** dimensions  # [-1, 1]^n has volume 2^n
    return cube_volume * inside / n_samples

# Known volumes for comparison
# V_2 = Pi (area of unit circle)
# V_3 = 4/3 * Pi (volume of unit sphere)
# V_4 = Pi^2 / 2

print("Unit sphere volume in n dimensions (1M samples)")
print(f"{'Dim':>5} {'Estimate':>12} {'Exact':>12} {'Error %':>10}")
print("-" * 42)

exact_volumes = {
    2: np.pi,
    3: 4/3 * np.pi,
    4: np.pi**2 / 2,
    5: 8 * np.pi**2 / 15
}

for dim in [2, 3, 4, 5]:
    estimate = integrate_sphere_volume(1000000, dim)
    exact = exact_volumes[dim]
    error_pct = abs(estimate - exact) / exact * 100
    print(f"{dim:>5} {estimate:>12.4f} {exact:>12.4f} {error_pct:>10.2f}")

Unit sphere volume in n dimensions (1M samples)
  Dim     Estimate        Exact    Error %
------------------------------------------
    2       3.1410       3.1416       0.02
    3       4.1868       4.1888       0.05
    4       4.9254       4.9348       0.19
    5       5.2565       5.2638       0.14


## 6. Performance Comparison

Compare HPXPy parallel performance with NumPy.

In [7]:
def benchmark_pi(n_samples, warmup=2, repeats=5):
    """Benchmark Pi estimation."""
    # Warmup
    for _ in range(warmup):
        estimate_pi_numpy(n_samples // 10)
        estimate_pi_hpxpy(n_samples // 10)
    
    # Time NumPy
    np_times = []
    for _ in range(repeats):
        start = time.perf_counter()
        estimate_pi_numpy(n_samples)
        np_times.append(time.perf_counter() - start)
    
    # Time HPXPy
    hpx_times = []
    for _ in range(repeats):
        start = time.perf_counter()
        estimate_pi_hpxpy(n_samples)
        hpx_times.append(time.perf_counter() - start)
    
    return min(np_times) * 1000, min(hpx_times) * 1000

print(f"Pi Estimation Performance (HPX threads: {hpx.num_threads()})")
print(f"{'Samples':>12} {'NumPy (ms)':>12} {'HPXPy (ms)':>12} {'Speedup':>10}")
print("=" * 50)

for n in [100000, 1000000, 10000000]:
    np_time, hpx_time = benchmark_pi(n)
    speedup = np_time / hpx_time if hpx_time > 0 else float('inf')
    print(f"{n:>12,} {np_time:>12.2f} {hpx_time:>12.2f} {speedup:>9.2f}x")

Pi Estimation Performance (HPX threads: 12)
     Samples   NumPy (ms)   HPXPy (ms)    Speedup
     100,000         0.93         1.32      0.70x


   1,000,000         9.45        11.22      0.84x


  10,000,000        93.97       107.53      0.87x


## 7. Cleanup

In [8]:
hpx.finalize()
print("HPX runtime finalized")

HPX runtime finalized


## Summary

Monte Carlo methods are naturally parallel - each random sample is independent. HPXPy leverages this with:

1. **Parallel random number generation** - `hpx.random.rand()` generates samples in parallel
2. **Parallel arithmetic** - Element-wise operations use HPX parallel algorithms
3. **Parallel reduction** - `hpx.sum()` and `hpx.mean()` use parallel reduce

### Best Practices

- Use large sample sizes (> 100K) to amortize parallel overhead
- Batch operations instead of loops
- Use vectorized operations for function evaluation

### Further Reading

- [Monte Carlo Methods in Finance](https://en.wikipedia.org/wiki/Monte_Carlo_methods_in_finance)
- [Importance Sampling](https://en.wikipedia.org/wiki/Importance_sampling)